PLM (Pretraining Language Model)

# **1.NLI(Natual Language Inference) 실습**
* 두 개의 문장(전제와 가설) 사이의 논리적 관계를 결정하는 자연어 처리 문제

In [1]:
!pip install transformers

* https://huggingface.co/Huffon/klue-roberta-base-nli 참고

In [2]:
# pipline: 자연어 처리 작업을 간단한 코드로 여러 작업을 한번에 지우너
# AutoTokenizer: 자동으로 적절한 토크나이저를
from transformers import pipeline, AutoTokenizer

In [3]:
classifier = pipeline(
    'text-classification',
    model = 'Huffon/klue-roberta-base-nli',  # 위 링크에서 복사해서 가저옴
    return_all_scores = True
)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/pipelines/text_classification.py:104: UserWarning: `return_all_scores` is now deprecated,  if want a similar functionality use `top_k=None` instead of `return_all_scores=True` or `top_k=1` instead of `return_all_scores=False`.
  warnings.warn(


In [4]:
tokenizer = AutoTokenizer.from_pretrained('Huffon/klue-roberta-base-nli')
tokenizer

BertTokenizerFast(name_or_path='Huffon/klue-roberta-base-nli', vocab_size=32000, model_max_length=512, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'unk_token': '[UNK]', 'sep_token': '[SEP]', 'pad_token': '[PAD]', 'cls_token': '[CLS]', 'mask_token': '[MASK]'}, clean_up_tokenization_spaces=True),  added_tokens_decoder={
	0: AddedToken("[PAD]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	1: AddedToken("[UNK]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	2: AddedToken("[CLS]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	3: AddedToken("[SEP]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	4: AddedToken("[MASK]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
}

In [5]:
tokenizer.sep_token

'[SEP]'

In [6]:
# ENTAILMENT: 첫 번째 문장이 두 번째 문장을 내포하거나 함축한 경우
# NEUTRAL: 두 문장의 관계가 특별히 긍정적이거나 부정적이지 않을 때(모순 관계가 없을 때)
# CONTRADICTION: 명벽한 모순 관계가 있을 때
classifier(f'나는 게임을 너무 좋아해 {tokenizer.sep_token} 나는 게임이 너무 싫어')

[[{'label': 'ENTAILMENT', 'score': 0.0003334383654873818},
  {'label': 'NEUTRAL', 'score': 0.000434514950029552},
  {'label': 'CONTRADICTION', 'score': 0.9992320537567139}]]

In [7]:
classifier(f'여러 남성들이 축구를 즐기고 있어요 {tokenizer.sep_token} 어떤 남자들은 공을 차고 있어요')

[[{'label': 'ENTAILMENT', 'score': 0.8508491516113281},
  {'label': 'NEUTRAL', 'score': 0.14858852326869965},
  {'label': 'CONTRADICTION', 'score': 0.0005623272736556828}]]

# **2. 문장 요약, 번역, 텍스트 생성 실습**
* BART(Sequential Bidirectional AutoRegressive Transformers)
* Hugging Face에서 개발한 자연어 처리 모델
* 자연어 생성 및 이해 작업을 위한 사전 훈련된 언어 모델
* Transformer 아키텍처를 기반으로 양벙향 인코더-디코더 구조를 가지고 있음
* 문장 요약, 번역 텍스트 생성에 뛰어난 성능을 보여주는 모델

* https://huggingface.co/digit82/kobart-summarization 참고

In [8]:
import torch
from transformers import PreTrainedTokenizerFast
from transformers import BartForConditionalGeneration

In [9]:
# PreTrainedTokenizerFast: 허깅페이스에서 개발한 Rust기반의 고성능 토크나이저
# 객체생성
tokenizer = PreTrainedTokenizerFast.from_pretrained('digit82/kobart-summarization')
tokenizer

PreTrainedTokenizerFast(name_or_path='digit82/kobart-summarization', vocab_size=30000, model_max_length=1000000000000000019884624838656, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'bos_token': '<s>', 'eos_token': '</s>', 'unk_token': '<unk>', 'pad_token': '<pad>', 'mask_token': '<mask>'}, clean_up_tokenization_spaces=True),  added_tokens_decoder={
	0: AddedToken("<s>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	1: AddedToken("</s>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	2: AddedToken("<usr>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	3: AddedToken("<pad>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	4: AddedToken("<sys>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	5: AddedToken("<unk>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=Tr

In [10]:
# 모델생성
model = BartForConditionalGeneration.from_pretrained('digit82/kobart-summarization')
model

You passed along `num_labels=3` with an incompatible id to label map: {'0': 'NEGATIVE', '1': 'POSITIVE'}. The number of labels wil be overwritten to 2.


BartForConditionalGeneration(
  (model): BartModel(
    (shared): Embedding(30000, 768, padding_idx=3)
    (encoder): BartEncoder(
      (embed_tokens): BartScaledWordEmbedding(30000, 768, padding_idx=3)
      (embed_positions): BartLearnedPositionalEmbedding(1028, 768)
      (layers): ModuleList(
        (0-5): 6 x BartEncoderLayer(
          (self_attn): BartSdpaAttention(
            (k_proj): Linear(in_features=768, out_features=768, bias=True)
            (v_proj): Linear(in_features=768, out_features=768, bias=True)
            (q_proj): Linear(in_features=768, out_features=768, bias=True)
            (out_proj): Linear(in_features=768, out_features=768, bias=True)
          )
          (self_attn_layer_norm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
          (activation_fn): GELUActivation()
          (fc1): Linear(in_features=768, out_features=3072, bias=True)
          (fc2): Linear(in_features=3072, out_features=768, bias=True)
          (final_layer_norm): Laye

In [11]:
text = '''
기후 관련 신기록들이 나오고 있다. 지난 6월 5일 '세계기상기구'(WMO)는 향후 5년간 산업혁명 전 대비 지구표면 온도가 1.1℃에서 1.9℃까지 상승할 수 있다고 발표했다. 기후위기의 마지노선인 1.5℃ 상승이 앞으로 5년간 계속될 확률도 47%라고 한다. 1.5℃ 상승제한을 결정했던 2015년 유엔 파리기후협정 당시만 해도 2030년까지 1.5℃ 오를 가능성은 0%였다. 10년도 안 되어 사실상 기후 관련 국제사회의 결정이 무너지고 있다.
지난 6월 28일 발생해 시속 270km의 강풍을 동반, 최고등급인 5등급으로 발달한 허리케인 '베릴'(Beryl)이 중남미 카리브해의 나라들을 파괴했다. 7월 3일 영국 BBC방송은 베릴에 대해 지난 100년간의 허리케인 기록을 깼고, 현재 진행되는 기후변화의 위험성이 집중 조명된 사례로 보도했다. 지금까지 허리케인은 해수면 온도가 높아진 8월 말부터 발생했는데, 6월에 발생한 허리케인 베릴은 기후위기가 만든 최초 기록이라는 것이다.
'''

In [12]:
text = text.replace('\n', ' ')

In [13]:
raw_input_ids = tokenizer.encode(text)
print(raw_input_ids)

[21009, 14342, 14094, 17469, 14108, 17427, 15964, 14141, 15328, 16415, 14063, 15666, 9264, 11224, 17842, 18223, 286, 276, 278, 14831, 16090, 22030, 14982, 22635, 14038, 15171, 17581, 13497, 10586, 14488, 14973, 14035, 15275, 370, 266, 14030, 14035, 16157, 370, 266, 14129, 14919, 13594, 14032, 14857, 14615, 15615, 21009, 11973, 15525, 14087, 24828, 11268, 12037, 25423, 370, 266, 14919, 12034, 15023, 22030, 28851, 14181, 10432, 9866, 20665, 236, 14218, 19553, 25423, 370, 266, 14919, 18010, 12007, 14657, 14622, 17655, 17650, 14240, 16287, 13756, 21761, 14616, 10500, 15766, 14182, 250, 14515, 14129, 25423, 370, 266, 22727, 21452, 1700, 14859, 20029, 16889, 9866, 14105, 15380, 16500, 21009, 14342, 14725, 20516, 26193, 18413, 14429, 15964, 14141, 15328, 16838, 27116, 28827, 15433, 247, 16413, 12024, 14119, 23073, 17554, 243, 14614, 17503, 12037, 14144, 17503, 14027, 19104, 13590, 22226, 12934, 12037, 14063, 10849, 10482, 18223, 265, 14879, 320, 307, 15019, 14059, 9506, 10746, 14471, 10476, 1

In [14]:
# 스페셜 토큰 붙이기
# 인코딩 붙이고, eos붙여서 시작과 끝 만듦
input_ids = [tokenizer.bos_token_id] + raw_input_ids + [tokenizer.eos_token_id]
print(input_ids)

[0, 21009, 14342, 14094, 17469, 14108, 17427, 15964, 14141, 15328, 16415, 14063, 15666, 9264, 11224, 17842, 18223, 286, 276, 278, 14831, 16090, 22030, 14982, 22635, 14038, 15171, 17581, 13497, 10586, 14488, 14973, 14035, 15275, 370, 266, 14030, 14035, 16157, 370, 266, 14129, 14919, 13594, 14032, 14857, 14615, 15615, 21009, 11973, 15525, 14087, 24828, 11268, 12037, 25423, 370, 266, 14919, 12034, 15023, 22030, 28851, 14181, 10432, 9866, 20665, 236, 14218, 19553, 25423, 370, 266, 14919, 18010, 12007, 14657, 14622, 17655, 17650, 14240, 16287, 13756, 21761, 14616, 10500, 15766, 14182, 250, 14515, 14129, 25423, 370, 266, 22727, 21452, 1700, 14859, 20029, 16889, 9866, 14105, 15380, 16500, 21009, 14342, 14725, 20516, 26193, 18413, 14429, 15964, 14141, 15328, 16838, 27116, 28827, 15433, 247, 16413, 12024, 14119, 23073, 17554, 243, 14614, 17503, 12037, 14144, 17503, 14027, 19104, 13590, 22226, 12934, 12037, 14063, 10849, 10482, 18223, 265, 14879, 320, 307, 15019, 14059, 9506, 10746, 14471, 10476

In [15]:
summary_ids = model.generate(torch.tensor([input_ids]),  num_beams=4,  max_length=512,  eos_token_id=1)
summary_ids

tensor([[    2, 14141, 15328, 16415, 14063, 15666,  9264, 11224, 17842, 18223,
           286,   276,   278, 14831, 16090, 22030, 14982, 22635, 14038, 15171,
         17581, 13497, 10586, 14488, 14973, 14035, 15275,   370,   266, 14030,
         14035, 16157,   370,   266, 14129, 14919, 13594, 14032, 14857, 14615,
         17432, 16090, 22030, 14982, 22635, 14038, 15171, 17581, 13497, 10586,
         14488, 14973, 14035, 15275,   370,   266, 14030, 14035, 16157,   370,
           266, 14129, 14919, 13594, 14032, 14857, 14615, 15615,     1]])

In [16]:
tokenizer.decode(summary_ids.squeeze().tolist(), skip_special_tokens=True)

"지난 6월 5일 '세계기상기구'(WMO)는 향후 5년간 산업혁명 전 대비 지구표면 온도가 1.1°C에서 1.9°C까지 상승할 수 있다고 발표했으며, 향후 5년간 산업혁명 전 대비 지구표면 온도가 1.1°C에서 1.9°C까지 상승할 수 있다고 발표했다."

# **3. KLUE(Korean Language Understanding Evaluation)**
* 한국어 자연어 이해 평가 데이터셋
* 한국어 언어모델의 공정한 평가를 위한 목적으로 8개의 종류가 포함된 공개 데이터셋
    * 뉴스 헤드라인 분류
    * 문장 유사도 비교
    * 자연어 추론
    * 개체명 인식
    * 관계 추출
    * 형태소 및 의존 구문 분석
    * 기계 독해 이해
    * 대화 상태 추적
* 광범위한 주제와 다양한 스타일을 포괄하기 위해 다양한 출처에서 공개적으로 사용 가능한 한국어 말뭉치를 수집
* 약 62GB 크기의 최종 사전 학습 코퍼스를 구축
* MODU(국립국어원), CC-100-Kor(CC-Net을 사용한 다국어 웹 크롤링), 나무위키(웹 기반 백과사전), 뉴스스크롤(뉴스 집계 플랫폼), 청원(청와대 국민청원의 기사) 등

* ['ynat', 'sts', 'nli', 'ner', 're', 'dp', 'mrc', 'wos']
    * ynat: 유튜브 비디오 댓글에서 자연스럽게 발생하는 대화 데이터를 이용한 태스크, 주어진 문장에 대해 답변하는 작업
    * sts: 두 텍스트의 의미적 유사성을 평가하는 태스크
    * nli: 전제와 가설이라는 두 문장 간의 논리적 관계를 판별하는 태스크(참, 거짓, 중립)
    * ner: 문장에서 인명, 지명, 기관명 등 특정 개체명을 식별하고 분류하는 태스크
    * re: 문장 또는 텍스트에서 개체들 간의 관계를 추출하는 태스크
        * 예) "스티브잡스는 애플의 공동 창립자이다" -> 스티브잡스와 애플을 뽑아내고 그 관계를 알려줌
    * dp: 문장 내 단어들 간의 문법적 관계를 파악하는 태스크. 각 단어가 어떻게 다른 단어들과 연결되어 있는지(종속성)를 구조적으로 분석
    * mrc: 주어진 텍스트와 질문에 대해 답을 추출하거나 생성하는 태스크. 모델이 텍스트를 읽고 이해하여 질문에 답할 ㅅ구 있도록 함
    * wos: Web of Science 데이터베이스에서 추출한 데이터를 활용하는 태스크. 특정 태스크보다는 데이터의 출처를 나타냄

In [17]:
!pip install datasets
!pip install accelerate -U
!pip install transformers[torch]

In [18]:
import datasets
import random
import pandas as pd
import numpy as np
from datasets import load_dataset, load_metric
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer

In [19]:
model_checkpoint = 'klue/roberta-base'
batch_size=64
task='ynat'

klue_datasets = load_dataset('klue', task)

In [20]:
klue_datasets

DatasetDict({
    train: Dataset({
        features: ['guid', 'title', 'label', 'url', 'date'],
        num_rows: 45678
    })
    validation: Dataset({
        features: ['guid', 'title', 'label', 'url', 'date'],
        num_rows: 9107
    })
})

In [21]:
klue_datasets['train'][:10]

{'guid': ['ynat-v1_train_00000',
  'ynat-v1_train_00001',
  'ynat-v1_train_00002',
  'ynat-v1_train_00003',
  'ynat-v1_train_00004',
  'ynat-v1_train_00005',
  'ynat-v1_train_00006',
  'ynat-v1_train_00007',
  'ynat-v1_train_00008',
  'ynat-v1_train_00009'],
 'title': ['유튜브 내달 2일까지 크리에이터 지원 공간 운영',
  '어버이날 맑다가 흐려져…남부지방 옅은 황사',
  '내년부터 국가RD 평가 때 논문건수는 반영 않는다',
  '김명자 신임 과총 회장 원로와 젊은 과학자 지혜 모을 것',
  '회색인간 작가 김동식 양심고백 등 새 소설집 2권 출간',
  '야외서 생방송 하세요…액션캠 전용 요금제 잇따라',
  '월드컵 태극전사 16강 전초기지 레오강 입성종합',
  '미세먼지 속 출근길',
  '왓츠앱稅 230원에 성난 레바논 민심…총리사퇴로 이어져종합2보',
  '베트남 경제 고성장 지속…2분기 GDP 6.71% 성장'],
 'label': [3, 3, 2, 2, 3, 0, 5, 3, 4, 4],
 'url': ['https://news.naver.com/main/read.nhn?mode=LS2D&mid=shm&sid1=105&sid2=227&oid=001&aid=0008508947',
  'https://news.naver.com/main/read.nhn?mode=LS2D&mid=shm&sid1=103&sid2=248&oid=001&aid=0008384783',
  'https://news.naver.com/main/read.nhn?mode=LS2D&mid=shm&sid1=105&sid2=228&oid=001&aid=0008254585',
  'https://news.naver.com/main/read.nhn?mode=LS2D&mid=sh

In [22]:
def show_random_elements(dataset, num_examples=10):
    picks = []

    for _ in range(num_examples):
        pick = random.randint(0, len(dataset)-1)
        while pick in picks:
            pick = random.randint(0, len(dataset)-1)

        picks.append(pick)
    df = pd.DataFrame(dataset[picks])
    return df

In [23]:
show_random_elements(klue_datasets['train'])

,guid,title,label,url,date
0,ynat-v1_train_42664,경복궁 달빛 아래서 한복 패션쇼…20일 한복의 날,3,https://news.naver.com/main/read.nhn?mode=LS2D...,2017.10.18. 오전 10:00
1,ynat-v1_train_08854,트럼프 이란 외무장관과 만나는 것 시기상조였다,4,https://news.naver.com/main/read.nhn?mode=LS2D...,2019.08.26. 오후 4:47
2,ynat-v1_train_00670,아이폰의 심장 만들어온 애플 칩디자이너 퇴사,0,https://news.naver.com/main/read.nhn?mode=LS2D...,2019.03.31. 오전 6:57
3,ynat-v1_train_25140,건국대 연구팀 식물이 위로 성장하는데 뿌리 유전자가 관여,0,https://news.naver.com/main/read.nhn?mode=LS2D...,2016.08.26. 오후 4:47
4,ynat-v1_train_00901,트럼프 中 거래 원해도 美 준비 안돼…이란 체제전환 요구안해,4,https://news.naver.com/main/read.nhn?mode=LS2D...,2019.05.27. 오후 5:09
5,ynat-v1_train_30051,주말 N 여행 호남권 영원한 사랑을 찾아서…구례 산수유꽃 봄 물결 출렁,3,https://news.naver.com/main/read.nhn?mode=LS2D...,2017.03.17. 오전 11:00
6,ynat-v1_train_27389,유비케어 1분기 영업이익 31억원…126.8% 증가,1,https://news.naver.com/main/read.nhn?mode=LS2D...,2019.04.19. 오후 1:51
7,ynat-v1_train_15390,시진핑 시리아내전 해결·팔 국가수립 중동서 외교 보폭 확대,4,https://news.naver.com/main/read.nhn?mode=LS2D...,2016.01.22. 오전 9:46
8,ynat-v1_train_00818,인권선언 70년 권력에 맞서 진실을 외쳐라 공연,2,https://news.naver.com/main/read.nhn?mode=LS2D...,2018.05.25. 오전 6:01
9,ynat-v1_train_31348,2019문화결산 거장급 연주자와 빅공연 잇따른 클래식·무용,3,https://news.naver.com/main/read.nhn?mode=LS2D...,2019.12.15. 오전 7:00


In [24]:
fake_preds = np.random.randint(0, 2, size=(64,))
fake_labels = np.random.randint(0, 2, size=(64,))
fake_preds, fake_labels

(array([1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 0, 1, 0, 1, 0, 1, 1, 0, 0, 1, 0, 1,
        1, 1, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 1, 1, 1, 0, 0, 1, 1, 0, 1, 0,
        0, 0, 0, 1, 0, 1, 1, 0, 1, 0, 1, 0, 0, 1, 0, 1, 1, 1, 1, 0]),
 array([0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0,
        0, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 1,
        1, 1, 0, 0, 0, 1, 1, 0, 1, 1, 1, 0, 0, 0, 1, 1, 1, 0, 0, 0]))

In [25]:
metric = load_metric('f1')

<ipython-input-25-b37dd255292e>:1: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  metric = load_metric('f1')


In [26]:
metric.compute(predictions=fake_preds, references=fake_labels)

{'f1': 0.3934426229508197}

In [27]:
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint, use_fast=True)

In [28]:
tokenizer('평생 처음 주민들의 충격 증언... 대한민국 곳곳 이상징후')

{'input_ids': [0, 5577, 3790, 3972, 2031, 2079, 5326, 8105, 18, 18, 18, 4892, 5844, 3658, 2976, 2158, 2], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}

In [29]:
print(tokenizer.cls_token_id, tokenizer.eos_token_id)

0 2


In [30]:
tokenizer

BertTokenizerFast(name_or_path='klue/roberta-base', vocab_size=32000, model_max_length=512, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'bos_token': '[CLS]', 'eos_token': '[SEP]', 'unk_token': '[UNK]', 'sep_token': '[SEP]', 'pad_token': '[PAD]', 'cls_token': '[CLS]', 'mask_token': '[MASK]'}, clean_up_tokenization_spaces=True),  added_tokens_decoder={
	0: AddedToken("[CLS]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	1: AddedToken("[PAD]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	2: AddedToken("[SEP]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	3: AddedToken("[UNK]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	4: AddedToken("[MASK]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
}

In [31]:
print(f"문장1: {klue_datasets['train'][0]['title']}")

문장1: 유튜브 내달 2일까지 크리에이터 지원 공간 운영


In [32]:
def preprocess_function(examples):
    return tokenizer(
        examples['title'],
        truncation=True, # 최대 길이를 초과할 경우 초과된 부분을 잘라냄
        return_token_type_ids=False
    )

In [33]:
preprocess_function(klue_datasets['train'][:5])

{'input_ids': [[0, 10637, 8474, 22, 2210, 2299, 2118, 28940, 3691, 4101, 3792, 2], [0, 24905, 1042, 4795, 19982, 2129, 121, 6904, 16311, 3, 14392, 2], [0, 4172, 3797, 3728, 2107, 2134, 3777, 904, 6022, 2332, 2113, 2259, 4523, 1380, 2259, 2062, 2], [0, 12417, 2155, 7840, 604, 2859, 3873, 11554, 2522, 1539, 2073, 8446, 6626, 18818, 575, 2], [0, 13203, 2179, 2366, 4197, 7551, 2096, 8542, 2088, 2353, 886, 1244, 4393, 2027, 22, 2207, 8189, 2]], 'attention_mask': [[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]]}

In [34]:
encoded_datasets = klue_datasets.map(preprocess_function, batched=True)

In [35]:
num_labels = 7
# AutoModelForSequenceClassification
# 시퀀스 분류 작업(예: 감정분석, 텍스트 분류)
# 다양한 모델(BERT, RoBERTa 등) 지원
model = AutoModelForSequenceClassification.from_pretrained(model_checkpoint, num_labels=num_labels)

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at klue/roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [36]:
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return metric.compute(predictions=predictions, references=labels, average='macro')

In [37]:
metric_name = 'f1'

args = TrainingArguments(
    'test-tc',
    # 평가전략: 에폭이 끝날 때마다 평가를 수행
    evaluation_strategy='epoch',
    # 모델 체크포인트를 저장. 매 에포크마다 저장
    save_strategy='epoch',
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=5,
    # 가중치 감소의 설정
    weight_decay=0.01,
    # 학습이 끝난 후 가장 좋은 성능을 보인 모델을 불러올지 여부
    load_best_model_at_end=True,
    # 최적의 모델을 결정할 때 사용할 메트릭을 지정
    metric_for_best_model=metric_name
)

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1474: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [38]:
trainer = Trainer(
    model,
    args,
    train_dataset = encoded_datasets['train'],
    eval_dataset = encoded_datasets['validation'],
    tokenizer = tokenizer,
    compute_metrics = compute_metrics
)

In [39]:
trainer.train()

Epoch,Training Loss,Validation Loss,F1
1,0.506500,0.395740,0.862930
2,0.326000,0.372101,0.865195
3,0.234200,0.377631,0.868543
4,0.198600,0.404639,0.869046
5,0.148200,0.428935,0.867518


TrainOutput(global_step=3570, training_loss=0.26564210175800057, metrics={'train_runtime': 1059.0125, 'train_samples_per_second': 215.663, 'train_steps_per_second': 3.371, 'total_flos': 2655593232808500.0, 'train_loss': 0.26564210175800057, 'epoch': 5.0})

In [40]:
trainer.evaluate()

{'eval_loss': 0.40463897585868835,
 'eval_f1': 0.8690461990356465,
 'eval_runtime': 11.2727,
 'eval_samples_per_second': 807.881,
 'eval_steps_per_second': 12.686,
 'epoch': 5.0}

In [41]:
best_model_checkpoint = trainer.state.best_model_checkpoint
best_model_checkpoint

'test-tc/checkpoint-2856'

In [42]:
classifier = pipeline(
    'text-classification',
    model = './test-tc/checkpoint-2142',
    return_all_scores=True
)

/usr/local/lib/python3.10/dist-packages/transformers/pipelines/text_classification.py:104: UserWarning: `return_all_scores` is now deprecated,  if want a similar functionality use `top_k=None` instead of `return_all_scores=True` or `top_k=1` instead of `return_all_scores=False`.
  warnings.warn(


In [43]:
'''
0: IT과학
1: 경제
2: 사회
3: 생활문화
4: 세계
5: 스포츠
6: 정치
'''
classifier('평생 처음 주민들의 충격 증언... 대한민국 곳곳 이상징후')

[[{'label': 'LABEL_0', 'score': 0.012105610221624374},
  {'label': 'LABEL_1', 'score': 0.0012863827869296074},
  {'label': 'LABEL_2', 'score': 0.9643608927726746},
  {'label': 'LABEL_3', 'score': 0.01460069790482521},
  {'label': 'LABEL_4', 'score': 0.0028239721432328224},
  {'label': 'LABEL_5', 'score': 0.0004231016500853002},
  {'label': 'LABEL_6', 'score': 0.004399271681904793}]]

In [44]:
classifier('미 핵자산 북핵 대응 첫 명문화…공동성명 채택 의미는?')

[[{'label': 'LABEL_0', 'score': 0.0009460877045057714},
  {'label': 'LABEL_1', 'score': 0.0034595842007547617},
  {'label': 'LABEL_2', 'score': 0.0037959208711981773},
  {'label': 'LABEL_3', 'score': 0.0008247809018939734},
  {'label': 'LABEL_4', 'score': 0.6766839027404785},
  {'label': 'LABEL_5', 'score': 0.0007646706071682274},
  {'label': 'LABEL_6', 'score': 0.3135250210762024}]]

In [45]:
classifier('탈주 이제훈·구교환 파워 통했나···100만 관객 수 돌파')

[[{'label': 'LABEL_0', 'score': 0.0004885277594439685},
  {'label': 'LABEL_1', 'score': 0.0017004961846396327},
  {'label': 'LABEL_2', 'score': 0.010500235483050346},
  {'label': 'LABEL_3', 'score': 0.9855858683586121},
  {'label': 'LABEL_4', 'score': 0.0005865542334504426},
  {'label': 'LABEL_5', 'score': 0.0006903948960825801},
  {'label': 'LABEL_6', 'score': 0.00044802427873946726}]]

### **1. 한국어 금융 뉴스 긍정, 부정 분류**
* https://github.com/ukairia777/finance_sentiment_corpus
* prediction('ChatGPT 등장으로 인공지능 스타트업들이 비상이 걸렸다')
* ['부정']

### **2. 다중 클래스 분류**
* https://github.com/kakaobrain/kor-nlu-datasets
* 두 문장을 입력받아 두 문장이 '얽힘', '중립', '모순'을 분류해주는 문제
* sent1 = '흡연자분들은 발코니가 있는 방이면 발코니에서 흡연하세요'
* sent2 = '이 건물은 흡연이 금지 됩니다'
* prediction(sent1, sent2)
* ['모순']

### **3. 한국어 혐오 발언 다중 레이블 분류**
* https://github.com/adlnlp/K-MHaS
* prediction('틀니들은 왜 그렇게 민폐를 끼치냐?')
* ['연령차별']